<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Predicting shots made per game by Kobe Bryant


---

In this lab you'll be using regularized regression penalties Ridge, Lasso, and Elastic Net to try and predict how many shots Kobe Bryant made per game in his career.

The Kobe shots dataset has hundreds of columns representing different characteristics of each basketball game. Fitting an ordinary linear regression using every predictor would dramatically overfit the model considering the limited number of observations (games) we have available. Furthermore, many of the predictors have significant multicollinearity. 

**Warning:** Some of these calculations are computationally expensive and may take a while to execute.  It may be worth while to only use a portion of the data to perform these calculations, especially if you have experienced kernel issues in the past.

### 1. Load packages and data

In [7]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import r2_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [8]:
kobe = pd.read_csv('../../../../resource-datasets/kobe_bryant/kobe_superwide_games.csv')

In [9]:
kobe.shape

(1558, 645)

In [10]:
kobe.head()

,SHOTS_MADE,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
2,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4
4,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,5


In [11]:
for col in kobe.columns: 
    print(col) 

SHOTS_MADE
AWAY_GAME
SEASON_OPPONENT:atl:1996-97
SEASON_OPPONENT:atl:1997-98
SEASON_OPPONENT:atl:1999-00
SEASON_OPPONENT:atl:2000-01
SEASON_OPPONENT:atl:2001-02
SEASON_OPPONENT:atl:2002-03
SEASON_OPPONENT:atl:2003-04
SEASON_OPPONENT:atl:2004-05
SEASON_OPPONENT:atl:2005-06
SEASON_OPPONENT:atl:2006-07
SEASON_OPPONENT:atl:2007-08
SEASON_OPPONENT:atl:2008-09
SEASON_OPPONENT:atl:2009-10
SEASON_OPPONENT:atl:2010-11
SEASON_OPPONENT:atl:2011-12
SEASON_OPPONENT:atl:2012-13
SEASON_OPPONENT:atl:2013-14
SEASON_OPPONENT:atl:2014-15
SEASON_OPPONENT:atl:2015-16
SEASON_OPPONENT:bkn:2012-13
SEASON_OPPONENT:bkn:2015-16
SEASON_OPPONENT:bos:1996-97
SEASON_OPPONENT:bos:1997-98
SEASON_OPPONENT:bos:1999-00
SEASON_OPPONENT:bos:2001-02
SEASON_OPPONENT:bos:2002-03
SEASON_OPPONENT:bos:2003-04
SEASON_OPPONENT:bos:2004-05
SEASON_OPPONENT:bos:2005-06
SEASON_OPPONENT:bos:2006-07
SEASON_OPPONENT:bos:2007-08
SEASON_OPPONENT:bos:2008-09
SEASON_OPPONENT:bos:2009-10
SEASON_OPPONENT:bos:2010-11
SEASON_OPPONENT:bos:2011-12

In [12]:
for inx in kobe.index:
    #print(inx) MEAN_SHOT_DISTANCE  MEAN_SHOT_ANGLE  CAREER_GAME_NUMBER
    #COMBINED_SHOT_TYPE:bank_shot  COMBINED_SHOT_TYPE:dunk  

SyntaxError: unexpected EOF while parsing (<ipython-input-12-f70f8dfbb4a0>, line 3)

### 2. Examine the data

- How many columns are there?
- Examine what the observations (rows) and columns represent.
- Why might regularization be particularly useful for this dataset?

In [13]:
for col in kobe.columns:
    print("col",col)

col SHOTS_MADE
col AWAY_GAME
col SEASON_OPPONENT:atl:1996-97
col SEASON_OPPONENT:atl:1997-98
col SEASON_OPPONENT:atl:1999-00
col SEASON_OPPONENT:atl:2000-01
col SEASON_OPPONENT:atl:2001-02
col SEASON_OPPONENT:atl:2002-03
col SEASON_OPPONENT:atl:2003-04
col SEASON_OPPONENT:atl:2004-05
col SEASON_OPPONENT:atl:2005-06
col SEASON_OPPONENT:atl:2006-07
col SEASON_OPPONENT:atl:2007-08
col SEASON_OPPONENT:atl:2008-09
col SEASON_OPPONENT:atl:2009-10
col SEASON_OPPONENT:atl:2010-11
col SEASON_OPPONENT:atl:2011-12
col SEASON_OPPONENT:atl:2012-13
col SEASON_OPPONENT:atl:2013-14
col SEASON_OPPONENT:atl:2014-15
col SEASON_OPPONENT:atl:2015-16
col SEASON_OPPONENT:bkn:2012-13
col SEASON_OPPONENT:bkn:2015-16
col SEASON_OPPONENT:bos:1996-97
col SEASON_OPPONENT:bos:1997-98
col SEASON_OPPONENT:bos:1999-00
col SEASON_OPPONENT:bos:2001-02
col SEASON_OPPONENT:bos:2002-03
col SEASON_OPPONENT:bos:2003-04
col SEASON_OPPONENT:bos:2004-05
col SEASON_OPPONENT:bos:2005-06
col SEASON_OPPONENT:bos:2006-07
col SEASON_

In [14]:
kobe.drop(kobe.accurate, inplace=True)


AttributeError: 'DataFrame' object has no attribute 'accurate'

In [ ]:
kobe.drop("accurate", axis=1, inplace=True)


In [15]:
kobe.columns


Index(['SHOTS_MADE', 'AWAY_GAME', 'SEASON_OPPONENT:atl:1996-97',
       'SEASON_OPPONENT:atl:1997-98', 'SEASON_OPPONENT:atl:1999-00',
       'SEASON_OPPONENT:atl:2000-01', 'SEASON_OPPONENT:atl:2001-02',
       'SEASON_OPPONENT:atl:2002-03', 'SEASON_OPPONENT:atl:2003-04',
       'SEASON_OPPONENT:atl:2004-05',
       ...
       'ACTION_TYPE:tip_layup_shot', 'ACTION_TYPE:tip_shot',
       'ACTION_TYPE:turnaround_bank_shot',
       'ACTION_TYPE:turnaround_fadeaway_bank_jump_shot',
       'ACTION_TYPE:turnaround_fadeaway_shot',
       'ACTION_TYPE:turnaround_finger_roll_shot',
       'ACTION_TYPE:turnaround_hook_shot', 'ACTION_TYPE:turnaround_jump_shot',
       'SEASON_GAME_NUMBER', 'CAREER_GAME_NUMBER'],
      dtype='object', length=645)

In [16]:
kobe_1 = kobe.copy()


In [17]:
kobe_1=pd.DataFrame(kobe_1)


In [18]:
kobe_1.head()


,SHOTS_MADE,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
2,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4
4,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,5


In [19]:
kobe_1.head().drop("SHOTS_MADE", axis=1, inplace=True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [20]:
kobe_1.head()

,SHOTS_MADE,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
2,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4
4,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,5


### 3. Make predictor and target variables. Standardize the predictors.

Why is normalization necessary for regularized regressions?

Use the `sklearn.preprocessing` class `StandardScaler` to standardize the predictors.

In [21]:
y = kobe.SHOTS_MADE.values

In [22]:
X = kobe.iloc[:,1:]

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1246, 644) (312, 644) (1246,) (312,)


In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std_train = scaler.fit_transform(X_train)
print('Mean:', X_std_train.mean(), 'Std:', X_std_train.std())


Mean: -7.010909581942456e-18 Std: 0.983560523988141


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [25]:
#X.mean()


### 4. Build a linear regression predicting `SHOTS_MADE` from the rest of the columns.

Cross-validate the $R^2$ of an ordinary linear regression model with 10 cross-validation folds.

How does it perform?

In [26]:
X.shape

(1558, 644)

In [27]:
X_std_train.shape

(1246, 644)

In [28]:
y.shape

(1558,)

In [29]:
X_train.shape

(1246, 644)

In [30]:
lr = LinearRegression()
lr.fit(X_std_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [31]:
print('Intercept:', lr.intercept_)
print('Coefs:', lr.coef_)

Intercept: 7.34736444851905
Coefs: [-1.77299844e-01  3.81754358e+11 -8.59062620e+11 -2.59106804e+12
  1.39067670e+12 -1.65060290e+12 -5.78123254e+12 -4.60185770e+12
  7.07044656e+12  3.74892895e+12  3.31383143e+12 -1.86508180e+12
  3.52469833e+11 -1.77004473e+12  1.72324710e+12 -6.11837762e+11
  2.09405602e+12 -3.16331793e+12  6.44321341e+12  2.46603464e+12
  2.09405602e+12  2.46603464e+12  2.70049571e+11 -1.21441000e+12
 -3.66285165e+12 -1.16762152e+12 -8.17261331e+12 -4.60185770e+12
  7.07044656e+12  3.74892895e+12  3.31383143e+12 -2.94539529e+12
  4.98267390e+11 -2.50120822e+12  1.72324710e+12 -6.11837762e+11
  1.48131624e+12  6.44321341e+12  2.46603464e+12  3.81754358e+11
 -1.21441000e+12 -1.74830827e+13 -2.59106804e+12  1.96592384e+12
 -1.65060290e+12  7.07044656e+12  3.74892895e+12  4.68458286e+12
 -1.31934195e+12  3.52469833e+11 -1.77004473e+12  1.21900937e+12
 -6.11837762e+11  1.48131624e+12 -3.16331793e+12  3.02200671e+13
  2.46603464e+12  3.81754358e+11 -8.59062620e+11 -2.591

In [32]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
# Perform 5-fold cross validation
scores = cross_val_score(lr, X_std_train,  y_train, cv=10)
print("Cross-validated scores:", scores)
print("Mean of Ccoss-validated scores:", scores.mean())


Cross-validated scores: [-1.70805852e+28 -3.14089785e+27 -4.73805157e+27 -3.81106002e+27
 -1.30678923e+28 -1.30903683e+28 -1.06557804e+28 -6.06465054e+27
 -1.10931277e+28 -1.34506639e+28]
Mean of Ccoss-validated scores: -9.619307780143324e+27


In [33]:
# only standardized data can go through CV, why?

### 5. Find an optimal value for Ridge regression alpha using `RidgeCV`.

[Go to the documentation and read how RidgeCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)

> *Hint: once the RidgeCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that Ridge performs best searching alphas through logarithmic space (`np.logspace`). This may take awhile to fit!


In [34]:
model = RidgeCV(alphas=np.logspace(-5, 5, 100), cv=5)
model.fit(X_std_train, y_train)
print(model.alpha_)

1519.9110829529332


In [35]:
ridge_alphas = np.logspace(0, 5, 200)

optimal_ridge = RidgeCV(alphas=ridge_alphas, cv=10)
optimal_ridge.fit(X_std_train, y_train)

print(optimal_ridge.alpha_)

1382.6221737646563


### 6. Cross-validate the Ridge regression $R^2$ with the optimal alpha.

Is it better than the Linear regression? If so, why might this be?

In [36]:
# A:

### 7. Find an optimal value for Lasso regression alpha using `LassoCV`.

[Go to the documentation and read how LassoCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) It is very similar to `RidgeCV`.

> *Hint: again, once the `LassoCV` is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that Lasso, unlike Ridge, performs best searching for alpha through linear space (`np.linspace`). However, you can actually let the LassoCV decide itself what alphas to use by instead setting the keyword argument `n_alphas=` to however many alphas you want it to search over. It is recommended to let sklearn choose the range of alphas.

_**Tip:** If you find your CV taking a long time and you're not sure if its working set `verbose =1`._

In [37]:
# A:

### 8. Cross-validate the Lasso $R^2$ with the optimal alpha.

Is it better than the Linear regression? Is it better than Ridge? What do the differences in results imply about the issues with the dataset?

In [38]:
# A:

### 9. Look at the coefficients for variables in the Lasso.

1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.
2. What percent of the variables in the original dataset are "zeroed-out" by the lasso?
3. What are the most important predictors for how many shots Kobe made in a game?

> **Note:** if you only fit the Lasso within `cross_val_score`, you will have to refit it outside of that
function to pull out the coefficients.

In [39]:
# A:

### 10. Find an optimal value for Elastic Net regression alpha using `ElasticNetCV`.

[Go to the documentation and read how ElasticNetCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html).

Note here that you will be optimizing both the alpha parameter and the l1_ratio:
- `alpha`: strength of regularization
- `l1_ratio`: amount of ridge vs. lasso (0 = all ridge, 1 = all lasso)
    
Do not include 0 in the search for `l1_ratio`: it will not allow it and break!

You can use `n_alphas` for the alpha parameters instead of setting your own values: highly recommended!

Also - be careful setting too many l1_ratios over cross-validation folds in your search. It can take a very long time if you choose too many combinations and for the most part there are diminishing returns in this data.

In [40]:
# A:

### 11. Cross-validate the ElasticNet $R^2$ with the optimal alpha and l1_ratio.

How does it compare to the Ridge and Lasso regularized regressions?

In [41]:
# A:

### 12. [Bonus] Compare the residuals for the Ridge and Lasso visually.


In [42]:
# A: Maybe a histogram?